# `xdggs`: using Discrete Global Grid Systems (DGGS) with `xarray`

<!-- rendered using jupyterlab-myst

Dependencies:
- xarray
- xdggs
- pint-xarray
- cmocean
-->

https://github.com/xarray-contrib/xdggs | https://xdggs.readthedocs.io

- Based on cell ids (for working with cell boundaries see e.g. `uxarray`)
- builtin support: healpix and H3, extensible by third-party libraries
- cell stat pages: [h3](https://h3geo.org/docs/core-library/restable)

```{warning}
Interpolation to DGGS is a mostly unsolved problem in python, in the following we assume we already have data on a DGGS grid.
```

```{image}  global-hexagons-3094508851.jpg
---
width: 400
height: 400
title: H3
---
H3
```
```{image}  gorski_f1-3990760175.jpg
---
width: 400
height: 400
title: Healpix
---
Healpix
```

In [ ]:
# calculate healpix stats page
import healpy as hp
import numpy as np
from rich.table import Table

table = Table()
table.add_column("level")
table.add_column("n_cells")
table.add_column("n_cells per base pixel")
table.add_column("area (degree²)", justify="right")
table.add_column("area (km²)", justify="right")
table.add_column("distance (degree)", justify="right")
table.add_column("distance (km)", justify="right")

r = 6371

for level in range(20):
    n_cells = 12 * 4**level
    area = 4 * np.pi / n_cells
    per_base_pixel = 4**level
    edge_length = np.sqrt(area)
    table.add_row(
        str(level),
        str(n_cells),
        str(per_base_pixel),
        f"{np.rad2deg(area):.012f}",
        f"{area * r**2:.012f}",
        f"{hp.nside2resol(2**level, arcmin=True) / 60:.012f}",
        f"{r * hp.nside2resol(2**level, arcmin=False):.012f}",
    )
table

## Setup

In [ ]:
import numpy as np
import xarray as xr
import xdggs

xr.set_options(display_expand_data=False)

In [ ]:
ds_healpix = xdggs.tutorial.open_dataset("air_temperature", "healpix").load()
ds_h3 = xdggs.tutorial.open_dataset("air_temperature", "h3").load()

display(ds_healpix, ds_h3)

In [ ]:
h3_coords = (
    ds_h3[["lat", "lon"]]
    .rename_vars({"lon": "longitude", "lat": "latitude"})
    .drop_vars("cell_ids")
    .drop_attrs()
)
healpix_coords = (
    ds_healpix[["lat", "lon"]]
    .rename_vars({"lon": "longitude", "lat": "latitude"})
    .drop_vars("cell_ids")
    .drop_attrs()
)

In [ ]:
healpix = ds_healpix.pipe(xdggs.decode)
h3 = ds_h3.pipe(xdggs.decode)

display(healpix, h3)

## Plotting

In [ ]:
import cmocean  # noqa: F401
import pint_xarray  # noqa: F401


def to_celsius(ds):
    return ds.pint.quantify().pint.to("degC").pint.dequantify().pipe(xdggs.decode)

In [ ]:
from ipywidgets import GridspecLayout

subset_h3 = h3["air"].isel(time=0)
subset_healpix = healpix["air"].isel(time=0)

subset_healpix.dggs.explore()

In [ ]:
grid = GridspecLayout(1, 2)
grid[0, 0] = subset_h3.dggs.explore(cmap="coolwarm", center=273.15, alpha=0.8)
grid[0, 1] = subset_healpix.dggs.explore(cmap="coolwarm", center=273.15, alpha=0.8)
grid

## Deriving data

### Cell centers

In [ ]:
h3_centers = h3.drop_vars(["lon", "lat"]).dggs.cell_centers()
h3_centers

In [ ]:
healpix_centers = healpix.drop_vars(["lon", "lat"]).dggs.cell_centers()
healpix_centers

In [ ]:
xr.testing.assert_allclose(h3_centers, h3_coords)
xr.testing.assert_allclose(healpix_centers, healpix_coords)

### Cell boundaries

In [ ]:
h3.dggs.cell_boundaries()

In [ ]:
healpix.dggs.cell_boundaries()

## Selecting points

In [ ]:
lats = np.array([40.0, 50.0, 60.0])
lons = np.array([230.0, 240.0, 250.0])

subset_healpix = healpix.dggs.sel_latlon(lats, lons)
subset_h3 = h3.dggs.sel_latlon(lats, lons)
display(subset_healpix, subset_h3)

In [ ]:
from ipywidgets import GridspecLayout

grid = GridspecLayout(1, 2)
grid[0, 0] = (
    subset_h3["air"]
    .isel(time=0)
    .copy(deep=True)
    .dggs.explore(cmap="coolwarm", center=273.15, alpha=0.8)
)
grid[0, 1] = (
    subset_healpix["air"]
    .isel(time=0)
    .copy(deep=True)
    .dggs.explore(cmap="coolwarm", center=273.15, alpha=0.8)
)
grid

## Future plans

- hierarchical operations: query parents, children, siblings
- neighbor search
- geometric selection: selection using geometric objects, for example by polygons / circles / lines / points
- automatic alignment
- ... and more!
